In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import SAGEConv, GAE
from torch.optim.lr_scheduler import StepLR
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
data = pd.read_csv("./data/train.csv")

In [ ]:
users = data[['User-ID', 'Age']].drop_duplicates().reset_index(drop=True)
books = data[['Book-ID', 'Year-Of-Publication']].drop_duplicates().reset_index(drop=True)

user_id_map = {user_id: idx for idx, user_id in enumerate(users['User-ID'].unique())}
book_id_map = {book_id: idx for idx, book_id in enumerate(books['Book-ID'].unique())}

data['Age'] = (users['Age'] - users['Age'].min()) / (users['Age'].max() - users['Age'].min())
data['Year-Of-Publication'] = (books['Year-Of-Publication'] - books['Year-Of-Publication'].min()) / (books['Year-Of-Publication'].max() - books['Year-Of-Publication'].min())

data['User-ID'] = data['User-ID'].map(user_id_map)
data['Book-ID'] = data['Book-ID'].map(book_id_map)

data_processed = data.fillna(0)

In [ ]:
node_features = data_processed[["Age","Year-Of-Publication"]].to_numpy()
edge_index = torch.tensor(data_processed[['User-ID', 'Book-ID']].values, dtype=torch.long).t().contiguous()

#################################### 더 적합한 것으로 추후 변경
edge_attr = torch.tensor(data_processed['Book-Rating'].values, dtype=torch.float).unsqueeze(-1)

target = torch.tensor(data_processed['Book-Rating'].values, dtype=torch.float)

In [ ]:
print(edge_index)
print(edge_index.shape)
print(f"Number of rows in data_processed: {len(data_processed)}")

# Make data

graph_data = Data(x=torch.tensor(node_features), edge_index=edge_index, edge_attr=edge_attr)
graph_data.n_id = torch.arange(graph_data.num_nodes)

In [ ]:
graph_data.x

In [ ]:
graph_data.edge_index

In [ ]:
graph_data.edge_attr

In [ ]:
graph_data.n_id

# Neighbor Loader